In [6]:
import boto3
import os

In [20]:
access_key = 'your_access_key'
secret_key = 'your_secret_key'
bucket_name = 'my-cfa-pdfs'
folder_key = 'pypdf/'
output_folder = './metadata_output/'

In [21]:
# Create an S3 client
# s3 = boto3.client('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_key)
s3 = boto3.client('s3')

In [22]:
try:
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_key)
    objects = response.get('Contents', [])

    if not objects:
        print("No files found in the specified folder.")
    else:
        # Create output folder if it doesn't exist
        os.makedirs(output_folder, exist_ok=True)

        # Save metadata to individual files
        print("Saving metadata to individual files:")
        for obj in objects:
            file_key = obj['Key']
            try:
                # Retrieve file metadata
                file_metadata = s3.head_object(Bucket=bucket_name, Key=file_key)
                
                # Extract metadata values
                last_modified = file_metadata['LastModified']
                etag = file_metadata['ETag']
                content_type = file_metadata.get('ContentType', 'N/A')
                content_length = file_metadata['ContentLength']
                s3_link = f"s3://{bucket_name}/{file_key}"

                # Create output file path
                output_file_path = os.path.join(output_folder, f"{file_key.replace('/', '_')}_metadata.txt")

                # Save metadata to file
                with open(output_file_path, 'w') as output_file:
                    output_file.write(f"Metadata for file: {file_key}\n")
                    output_file.write(f"  Last Modified: {last_modified}\n")
                    output_file.write(f"  ETag: {etag}\n")
                    output_file.write(f"  Content Type: {content_type}\n")
                    output_file.write(f"  Content Length: {content_length} bytes\n")
                    output_file.write(f"  S3 Link: {s3_link}\n")

                print(f"Metadata saved to {output_file_path}")

            except Exception as e:
                print(f"Error retrieving metadata for {file_key}: {e}")

except Exception as e:
    print(f"Error listing objects: {e}")

Saving metadata to individual files:
Metadata saved to ./metadata_output/pypdf__metadata.txt
Metadata saved to ./metadata_output/pypdf_2024-l1-topics-combined-2.txt_metadata.txt
Metadata saved to ./metadata_output/pypdf_2024-l1-topics-combined-2_extracted_text.txt_metadata.txt
Metadata saved to ./metadata_output/pypdf_2024-l2-topics-combined-2.txt_metadata.txt
Metadata saved to ./metadata_output/pypdf_2024-l2-topics-combined-2_extracted_text.txt_metadata.txt
Metadata saved to ./metadata_output/pypdf_2024-l3-topics-combined-2.txt_metadata.txt
Metadata saved to ./metadata_output/pypdf_2024-l3-topics-combined-2_extracted_text.txt_metadata.txt
Metadata saved to ./metadata_output/pypdf_coveo_links.txt_metadata.txt


In [2]:
import csv
import os
import boto3

# Replace 'your_access_key', 'your_secret_key', 'your_bucket_name', 'your_folder', and 'output_csv_file' with your AWS credentials and details
access_key = 'your_access_key'
secret_key = 'your_secret_key'
bucket_name = 'my-cfa-pdfs'
folder_key = 'pypdf/'  # Specify the folder path
output_csv_file = '../sample_output/metadata_output.csv'  # Specify the output CSV file path

# Create an S3 client
s3 = boto3.client('s3')

# List objects in the folder
try:
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_key)
    objects = response.get('Contents', [])

    if not objects:
        print("No files found in the specified folder.")
    else:
        # Organize metadata into a dictionary
        metadata_dict = {}

        # Iterate over objects and extract metadata
        for obj in objects:
            file_key = obj['Key']
            try:
                # Retrieve file metadata
                file_metadata = s3.head_object(Bucket=bucket_name, Key=file_key)

                # Extract common substring (e.g., '2024_l1_combined' or '2024_l2_combined')
                common_substring = file_key.split('_RR_')[1].split('_combined')[0]

                # If common substring not in dictionary, create an entry
                if common_substring not in metadata_dict:
                    metadata_dict[common_substring] = {}

                # Extract metadata values
                metadata_dict[common_substring][file_key] = {
                    'Last Modified': file_metadata['LastModified'],
                    'ETag': file_metadata['ETag'],
                    'Content Type': file_metadata.get('ContentType', 'N/A'),
                    'Content Length': file_metadata['ContentLength'],
                    'S3 Link': f"s3://{bucket_name}/{file_key}"
                }

            except Exception as e:
                print(f"Error retrieving metadata for {file_key}: {e}")

        # Save metadata to CSV file
        with open(output_csv_file, 'w', newline='', encoding='utf-8') as csv_file:
            csv_writer = csv.writer(csv_file)

            # Write header row
            header_row = ['Common Substring', 'File Type']
            for file_key, metadata in metadata_dict.items():
                for subfile_key, subfile_metadata in metadata.items():
                    for metadata_key in subfile_metadata.keys():
                        header_row.append(f"{file_key}_{metadata_key}")
            csv_writer.writerow(header_row)

            # Write data rows
            for common_substring, metadata in metadata_dict.items():
                for subfile_key, subfile_metadata in metadata.items():
                    data_row = [common_substring, subfile_key.split('_RR_')[0]]
                    for metadata_key, value in subfile_metadata.items():
                        data_row.extend([value['Last Modified'], value['ETag'], value['Content Type'], value['Content Length'], value['S3 Link']])
                    csv_writer.writerow(data_row)

        print(f"Metadata saved to {output_csv_file}")

except Exception as e:
    print(f"Error listing objects: {e}")


Error retrieving metadata for pypdf/: list index out of range
Error listing objects: 'datetime.datetime' object is not subscriptable


In [12]:
import csv
import os
import boto3
from datetime import datetime

# Replace 'your_access_key', 'your_secret_key', 'your_bucket_name', 'your_folder', and 'output_csv_file' with your AWS credentials and details
access_key = 'your_access_key'
secret_key = 'your_secret_key'
bucket_name = 'my-cfa-pdfs'
folder_key = 'pypdf/'  # Specify the folder path
output_csv_file = '../sample_output/metadata_output.csv'  # Specify the output CSV file path

# Create an S3 client
s3 = boto3.client('s3')

# List objects in the folder
try:
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_key)
    objects = response.get('Contents', [])

    if not objects:
        print("No files found in the specified folder.")
    else:
        # Save metadata to CSV file
        with open(output_csv_file, 'w', newline='', encoding='utf-8') as csv_file:
            csv_writer = csv.writer(csv_file)
            
            # Write header row
            csv_writer.writerow(['File Key', 'Last Modified', 'ETag', 'Content Type', 'Content Length', 'S3 Link'])
            print(objects)
            # Extract unique file keys (e.g., '2024_l1_combined', '2024_l2_combined', etc.)
            unique_file_keys = set(file['Key'].split('_')[3] for file in objects if file['Key'] != 'pypdf/')
            print("hello")
            # Write metadata for each unique file key
            for file_key_suffix in sorted(unique_file_keys):
                pypdf_key = f'PyPDF_RR_2024_l{file_key_suffix}_combined.txt'
                grobid_key = f'Grobid_RR_2024_l{file_key_suffix}_combined.txt'

                try:
                    # Retrieve metadata for PyPDF file
                    pypdf_metadata = s3.head_object(Bucket=bucket_name, Key=pypdf_key)
                    pypdf_last_modified = pypdf_metadata['LastModified']
                    pypdf_etag = pypdf_metadata['ETag']
                    pypdf_content_type = pypdf_metadata.get('ContentType', 'N/A')
                    pypdf_content_length = pypdf_metadata['ContentLength']
                    pypdf_s3_link = f"s3://{bucket_name}/{pypdf_key}"

                    # Retrieve metadata for Grobid file
                    grobid_metadata = s3.head_object(Bucket=bucket_name, Key=grobid_key)
                    grobid_last_modified = grobid_metadata['LastModified']
                    grobid_etag = grobid_metadata['ETag']
                    grobid_content_type = grobid_metadata.get('ContentType', 'N/A')
                    grobid_content_length = grobid_metadata['ContentLength']
                    grobid_s3_link = f"s3://{bucket_name}/{grobid_key}"
                    print([f'RR_2024_l{file_key_suffix}_combined',
                                         pypdf_last_modified, pypdf_etag, pypdf_content_type, pypdf_content_length, pypdf_s3_link,
                                         grobid_last_modified, grobid_etag, grobid_content_type, grobid_content_length, grobid_s3_link])
                    # Write metadata to CSV file
                    csv_writer.writerow([f'RR_2024_l{file_key_suffix}_combined',
                                         pypdf_last_modified, pypdf_etag, pypdf_content_type, pypdf_content_length, pypdf_s3_link,
                                         grobid_last_modified, grobid_etag, grobid_content_type, grobid_content_length, grobid_s3_link])

                except Exception as e:
                    print(f"Error retrieving metadata for file key {file_key_suffix}: {e}")

        print(f"Metadata saved to {output_csv_file}")

except Exception as e:
    print(f"Error listing objects: {e}")


[{'Key': 'pypdf/', 'LastModified': datetime.datetime(2024, 2, 11, 17, 33, 15, tzinfo=tzutc()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'Size': 0, 'StorageClass': 'STANDARD'}, {'Key': 'pypdf/Grobid_RR_2024_l1_combined.txt', 'LastModified': datetime.datetime(2024, 2, 13, 22, 21, 50, tzinfo=tzutc()), 'ETag': '"20923b87e7442a82db6c2901d9157931"', 'Size': 52416, 'StorageClass': 'STANDARD'}, {'Key': 'pypdf/Grobid_RR_2024_l2_combined.txt', 'LastModified': datetime.datetime(2024, 2, 13, 22, 21, 50, tzinfo=tzutc()), 'ETag': '"cd30b300f521c7ca1af3b379b4d63023"', 'Size': 50368, 'StorageClass': 'STANDARD'}, {'Key': 'pypdf/Grobid_RR_2024_l3_combined.txt', 'LastModified': datetime.datetime(2024, 2, 13, 22, 21, 50, tzinfo=tzutc()), 'ETag': '"97a9b17d373ba89ae1be0b47a33e9a5b"', 'Size': 32699, 'StorageClass': 'STANDARD'}, {'Key': 'pypdf/PyPDF_RR_2024_l1_combined.txt', 'LastModified': datetime.datetime(2024, 2, 13, 22, 21, 37, tzinfo=tzutc()), 'ETag': '"fd57ef7f8805d0191c7109f21d11837e"', 'Size': 

In [13]:
import csv
from collections import defaultdict
import boto3

# Replace 'your_access_key', 'your_secret_key', 'your_bucket_name', 'your_folder', and 'output_csv_file' with your AWS credentials and details
access_key = 'your_access_key'
secret_key = 'your_secret_key'
bucket_name = 'my-cfa-pdfs'
folder_key = 'pypdf/'  # Specify the folder path
output_csv_file = '../sample_output/metadata_output.csv'  # Specify the output CSV file path

# Create an S3 client
s3 = boto3.client('s3')

# List objects in the folder
try:
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_key)
    objects = response.get('Contents', [])

    if not objects:
        print("No files found in the specified folder.")
    else:
        # Create dictionaries to store metadata for PyPDF and Grobid separately
        pypdf_metadata = defaultdict(dict)
        grobid_metadata = defaultdict(dict)

        # Process each object and store metadata
        for obj in objects:
            file_key = obj['Key']
            if 'Grobid' in file_key:
                row_name = file_key.split('_l')[1].split('_combined')[0]  # Extract RR_2024_l1 from the key
                grobid_metadata[row_name]['File Key'] = file_key
                grobid_metadata[row_name]['Last Modified'] = obj['LastModified']
                grobid_metadata[row_name]['ETag'] = obj['ETag']
                grobid_metadata[row_name]['Size'] = obj['Size']
            elif 'PyPDF' in file_key:
                row_name = file_key.split('_l')[1].split('_combined')[0]  # Extract RR_2024_l1 from the key
                pypdf_metadata[row_name]['File Key'] = file_key
                pypdf_metadata[row_name]['Last Modified'] = obj['LastModified']
                pypdf_metadata[row_name]['ETag'] = obj['ETag']
                pypdf_metadata[row_name]['Size'] = obj['Size']

        # Combine PyPDF and Grobid metadata into rows
        rows = []
        for row_name, grobid_info in grobid_metadata.items():
            row = grobid_info
            row.update(pypdf_metadata.get(row_name, {}))
            rows.append(row)

        # Save metadata to CSV file
        with open(output_csv_file, 'w', newline='', encoding='utf-8') as csv_file:
            fieldnames = ['File Key', 'Last Modified', 'ETag', 'Size']  # Define the field names
            csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

            # Write header row
            csv_writer.writeheader()

            # Write metadata for each row
            for row in rows:
                csv_writer.writerow(row)

        print(f"Metadata saved to {output_csv_file}")

except Exception as e:
    print(f"Error listing objects: {e}")


Metadata saved to ../sample_output/metadata_output.csv


In [17]:
import csv
from collections import defaultdict
import boto3

# Replace 'your_access_key', 'your_secret_key', 'your_bucket_name', 'your_folder', and 'output_csv_file' with your AWS credentials and details
access_key = 'your_access_key'
secret_key = 'your_secret_key'
bucket_name = 'my-cfa-pdfs'
folder_key = 'pypdf/'  # Specify the folder path
output_csv_file = '../sample_output/metadata_output.csv'  # Specify the output CSV file path

# Create an S3 client
s3 = boto3.client('s3')

# List objects in the folder
try:
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_key)
    objects = response.get('Contents', [])

    if not objects:
        print("No files found in the specified folder.")
    else:
        # Create dictionaries to store metadata for PyPDF and Grobid separately
        pypdf_metadata = defaultdict(dict)
        grobid_metadata = defaultdict(dict)

        # Process each object and store metadata
        for obj in objects:
            file_key = obj['Key']
            if 'Grobid' in file_key:
                row_name = file_key.split('_l')[1].split('_combined')[0]  # Extract RR_2024_l1 from the key
                grobid_metadata[row_name]['File Key'] = file_key
                grobid_metadata[row_name]['Last Modified'] = obj['LastModified']
                grobid_metadata[row_name]['ETag'] = obj['ETag']
                grobid_metadata[row_name]['Size'] = obj['Size']
            elif 'PyPDF' in file_key:
                row_name = file_key.split('_l')[1].split('_combined')[0]  # Extract RR_2024_l1 from the key
                pypdf_metadata[row_name]['File Key'] = file_key
                pypdf_metadata[row_name]['Last Modified'] = obj['LastModified']
                pypdf_metadata[row_name]['ETag'] = obj['ETag']
                pypdf_metadata[row_name]['Size'] = obj['Size']

        # Combine PyPDF and Grobid metadata into rows
        rows = []
        for row_name, grobid_info in grobid_metadata.items():
            row = {}
            row.update(grobid_info)
            row.update(pypdf_metadata.get(row_name, {}))
            rows.append(row)

        # Save metadata to CSV file
        with open(output_csv_file, 'w', newline='', encoding='utf-8') as csv_file:
            fieldnames = ['File Key', 'Last Modified (Grobid)', 'ETag (Grobid)', 'Size (Grobid)',
                          'Last Modified (PyPDF)', 'ETag (PyPDF)', 'Size (PyPDF)']  # Define the field names
            csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

            # Write header row
            csv_writer.writeheader()

            # Write metadata for each row
            for row in rows:
                csv_writer.writerow({
                    'File Key': row.get('File Key', ''),
                    'Last Modified (Grobid)': row.get('Last Modified', ''),
                    'ETag (Grobid)': row.get('ETag', ''),
                    'Size (Grobid)': row.get('Size', ''),
                    'Last Modified (PyPDF)': row.get('Last Modified', ''),
                    'ETag (PyPDF)': row.get('ETag', ''),
                    'Size (PyPDF)': row.get('Size', ''),
                })

        print(f"Metadata saved to {output_csv_file}")

except Exception as e:
    print(f"Error listing objects: {e}")


Metadata saved to ../sample_output/metadata_output.csv


In [18]:
import csv
from collections import defaultdict
import boto3

# Replace 'your_access_key', 'your_secret_key', 'your_bucket_name', 'your_folder', and 'output_csv_file' with your AWS credentials and details
access_key = 'your_access_key'
secret_key = 'your_secret_key'
bucket_name = 'my-cfa-pdfs'
folder_key = 'pypdf/'  # Specify the folder path
output_csv_file = '../sample_output/metadata_output.csv'  # Specify the output CSV file path

# Create an S3 client
s3 = boto3.client('s3')

# List objects in the folder
try:
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_key)
    objects = response.get('Contents', [])

    if not objects:
        print("No files found in the specified folder.")
    else:
        # Create dictionaries to store metadata for PyPDF and Grobid separately
        pypdf_metadata = defaultdict(dict)
        grobid_metadata = defaultdict(dict)

        # Process each object and store metadata
        for obj in objects:
            file_key = obj['Key']
            if 'Grobid' in file_key:
                row_name = file_key.split('_l')[1].split('_combined')[0]  # Extract RR_2024_l1 from the key
                grobid_metadata[row_name]['File Key'] = row_name
                grobid_metadata[row_name]['Last Modified'] = obj['LastModified']
                grobid_metadata[row_name]['ETag'] = obj['ETag']
                grobid_metadata[row_name]['Size'] = obj['Size']
            elif 'PyPDF' in file_key:
                row_name = file_key.split('_l')[1].split('_combined')[0]  # Extract RR_2024_l1 from the key
                pypdf_metadata[row_name]['File Key'] = row_name
                pypdf_metadata[row_name]['Last Modified'] = obj['LastModified']
                pypdf_metadata[row_name]['ETag'] = obj['ETag']
                pypdf_metadata[row_name]['Size'] = obj['Size']

        # Combine PyPDF and Grobid metadata into rows
        rows = []
        for row_name, grobid_info in grobid_metadata.items():
            row = {}
            row.update(grobid_info)
            row.update(pypdf_metadata.get(row_name, {}))
            rows.append(row)

        # Save metadata to CSV file
        with open(output_csv_file, 'w', newline='', encoding='utf-8') as csv_file:
            fieldnames = ['File Key', 'Last Modified (Grobid)', 'ETag (Grobid)', 'Size (Grobid)',
                          'Last Modified (PyPDF)', 'ETag (PyPDF)', 'Size (PyPDF)']  # Define the field names
            csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

            # Write header row
            csv_writer.writeheader()

            # Write metadata for each row
            for row in rows:
                csv_writer.writerow({
                    'File Key': row.get('File Key', ''),
                    'Last Modified (Grobid)': row.get('Last Modified', ''),
                    'ETag (Grobid)': row.get('ETag', ''),
                    'Size (Grobid)': row.get('Size', ''),
                    'Last Modified (PyPDF)': row.get('Last Modified', ''),
                    'ETag (PyPDF)': row.get('ETag', ''),
                    'Size (PyPDF)': row.get('Size', ''),
                })

        print(f"Metadata saved to {output_csv_file}")

except Exception as e:
    print(f"Error listing objects: {e}")


Metadata saved to ../sample_output/metadata_output.csv


In [20]:
import csv
from collections import defaultdict
import boto3

# Replace 'your_access_key', 'your_secret_key', 'your_bucket_name', 'your_folder', and 'output_csv_file' with your AWS credentials and details
access_key = 'your_access_key'
secret_key = 'your_secret_key'
bucket_name = 'my-cfa-pdfs'
folder_key = 'pypdf/'  # Specify the folder path
output_csv_file = '../sample_output/metadata_output.csv'  # Specify the output CSV file path

# Create an S3 client
s3 = boto3.client('s3')

# List objects in the folder
try:
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_key)
    objects = response.get('Contents', [])

    if not objects:
        print("No files found in the specified folder.")
    else:
        # Create dictionaries to store metadata for PyPDF and Grobid separately
        pypdf_metadata = defaultdict(dict)
        grobid_metadata = defaultdict(dict)

        # Process each object and store metadata
        for obj in objects:
            file_key = obj['Key']
            if 'Grobid' in file_key:
                row_name = file_key.split('_', 1)[1]  # Extract RR_2024_l1 from the key
                grobid_metadata[row_name]['File Key'] = row_name  # Extract only RR_2024_l1_combined.txt
                grobid_metadata[row_name]['Last Modified'] = obj['LastModified']
                grobid_metadata[row_name]['ETag'] = obj['ETag']
                grobid_metadata[row_name]['Size'] = obj['Size']
            elif 'PyPDF' in file_key:
                row_name = file_key.split('_', 1)[1]  # Extract RR_2024_l1 from the key
                pypdf_metadata[row_name]['File Key'] = row_name  # Extract only RR_2024_l1_combined.txt
                pypdf_metadata[row_name]['Last Modified'] = obj['LastModified']
                pypdf_metadata[row_name]['ETag'] = obj['ETag']
                pypdf_metadata[row_name]['Size'] = obj['Size']

        # Combine PyPDF and Grobid metadata into rows
        rows = []
        for row_name, grobid_info in grobid_metadata.items():
            row = {}
            row.update(grobid_info)
            row.update(pypdf_metadata.get(row_name, {}))
            rows.append(row)

        # Save metadata to CSV file
        with open(output_csv_file, 'w', newline='', encoding='utf-8') as csv_file:
            fieldnames = ['File Key', 'Last Modified (Grobid)', 'ETag (Grobid)', 'Size (Grobid)',
                          'Last Modified (PyPDF)', 'ETag (PyPDF)', 'Size (PyPDF)']  # Define the field names
            csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

            # Write header row
            csv_writer.writeheader()

            # Write metadata for each row
            for row in rows:
                csv_writer.writerow({
                    'File Key': row.get('File Key', ''),
                    'Last Modified (Grobid)': row.get('Last Modified', ''),
                    'ETag (Grobid)': row.get('ETag', ''),
                    'Size (Grobid)': row.get('Size', ''),
                    'Last Modified (PyPDF)': row.get('Last Modified', ''),
                    'ETag (PyPDF)': row.get('ETag', ''),
                    'Size (PyPDF)': row.get('Size', ''),
                })

        print(f"Metadata saved to {output_csv_file}")

except Exception as e:
    print(f"Error listing objects: {e}")


Metadata saved to ../sample_output/metadata_output.csv


## Metadata Final

In [22]:
import csv
from collections import defaultdict
import boto3

# Replace 'your_access_key', 'your_secret_key', 'your_bucket_name', 'your_folder', and 'output_csv_file' with your AWS credentials and details
access_key = 'your_access_key'
secret_key = 'your_secret_key'
bucket_name = 'my-cfa-pdfs'
folder_key = 'pypdf/'  # Specify the folder path
output_csv_file = '../sample_output/metadata_output.csv'  # Specify the output CSV file path

# Create an S3 client
s3 = boto3.client('s3')

# List objects in the folder
try:
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_key)
    objects = response.get('Contents', [])

    if not objects:
        print("No files found in the specified folder.")
    else:
        # Create dictionaries to store metadata for PyPDF and Grobid separately
        pypdf_metadata = defaultdict(dict)
        grobid_metadata = defaultdict(dict)

        # Process each object and store metadata
        for obj in objects:
            file_key = obj['Key']
            if 'Grobid' in file_key:
                row_name = file_key.split('_', 1)[1]  # Extract RR_2024_l1 from the key
                grobid_metadata[row_name]['File Key'] = row_name  # Extract only RR_2024_l1_combined.txt
                grobid_metadata[row_name]['Last Modified'] = obj['LastModified']
                grobid_metadata[row_name]['ETag'] = obj['ETag']
                grobid_metadata[row_name]['Size'] = obj['Size']
                grobid_metadata[row_name]['S3 Link'] = f"https://{bucket_name}.s3.amazonaws.com/{file_key}"
                grobid_metadata[row_name]['File Type'] = file_key.split('.')[-1].lower()
            elif 'PyPDF' in file_key:
                row_name = file_key.split('_', 1)[1]  # Extract RR_2024_l1 from the key
                pypdf_metadata[row_name]['File Key'] = row_name  # Extract only RR_2024_l1_combined.txt
                pypdf_metadata[row_name]['Last Modified'] = obj['LastModified']
                pypdf_metadata[row_name]['ETag'] = obj['ETag']
                pypdf_metadata[row_name]['Size'] = obj['Size']
                pypdf_metadata[row_name]['S3 Link'] = f"https://{bucket_name}.s3.amazonaws.com/{file_key}"
                pypdf_metadata[row_name]['File Type'] = file_key.split('.')[-1].lower()

        # Combine PyPDF and Grobid metadata into rows
        rows = []
        for row_name, grobid_info in grobid_metadata.items():
            row = {}
            row.update(grobid_info)
            row.update(pypdf_metadata.get(row_name, {}))
            rows.append(row)

        # Save metadata to CSV file
        with open(output_csv_file, 'w', newline='', encoding='utf-8') as csv_file:
            fieldnames = ['File Key', 'Last Modified (Grobid)', 'ETag (Grobid)', 'Size (Grobid)', 'S3 Link (Grobid)', 'File Type (Grobid)',
                          'Last Modified (PyPDF)', 'ETag (PyPDF)', 'Size (PyPDF)', 'S3 Link (PyPDF)', 'File Type (PyPDF)']  # Define the field names
            csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

            # Write header row
            csv_writer.writeheader()

            # Write metadata for each row
            for row in rows:
                csv_writer.writerow({
                    'File Key': row.get('File Key', ''),
                    'Last Modified (Grobid)': row.get('Last Modified', ''),
                    'ETag (Grobid)': row.get('ETag', ''),
                    'Size (Grobid)': row.get('Size', ''),
                    'S3 Link (Grobid)': row.get('S3 Link', ''),
                    'File Type (Grobid)': row.get('File Type', ''),
                    'Last Modified (PyPDF)': row.get('Last Modified', ''),
                    'ETag (PyPDF)': row.get('ETag', ''),
                    'Size (PyPDF)': row.get('Size', ''),
                    'S3 Link (PyPDF)': row.get('S3 Link', ''),
                    'File Type (PyPDF)': row.get('File Type', ''),
                })

        print(f"Metadata saved to {output_csv_file}")

except Exception as e:
    print(f"Error listing objects: {e}")


Metadata saved to ../sample_output/metadata_output.csv


In [23]:
import csv
from collections import defaultdict
import boto3

# Replace 'your_access_key', 'your_secret_key', 'your_bucket_name', 'output_csv_file' with your AWS credentials and details
access_key = 'your_access_key'
secret_key = 'your_secret_key'
bucket_name = 'my-cfa-pdfs'
output_csv_file = '../sample_output/metadata_output.csv'

# Create an S3 client
s3 = boto3.client('s3')

# List objects for pypdf folder
pypdf_folder_key = 'pypdf/'
try:
    response_pypdf = s3.list_objects_v2(Bucket=bucket_name, Prefix=pypdf_folder_key)
    pypdf_objects = response_pypdf.get('Contents', [])
except Exception as e:
    print(f"Error listing pypdf objects: {e}")
    pypdf_objects = []

# List objects for grobid folder
grobid_folder_key = 'grobid/'
try:
    response_grobid = s3.list_objects_v2(Bucket=bucket_name, Prefix=grobid_folder_key)
    grobid_objects = response_grobid.get('Contents', [])
except Exception as e:
    print(f"Error listing grobid objects: {e}")
    grobid_objects = []

# Combine objects from both folders
objects = pypdf_objects + grobid_objects

# Process objects and store metadata
pypdf_metadata = defaultdict(dict)
grobid_metadata = defaultdict(dict)

for obj in objects:
    file_key = obj['Key']
    if 'Grobid' in file_key:
        row_name = file_key.split('_', 1)[1]
        grobid_metadata[row_name]['File Key'] = row_name
        grobid_metadata[row_name]['Last Modified'] = obj['LastModified']
        grobid_metadata[row_name]['ETag'] = obj['ETag']
        grobid_metadata[row_name]['Size'] = obj['Size']
        grobid_metadata[row_name]['S3 Link'] = f"https://{bucket_name}.s3.amazonaws.com/{file_key}"
        grobid_metadata[row_name]['File Type'] = file_key.split('.')[-1].lower()
    elif 'PyPDF' in file_key:
        row_name = file_key.split('_', 1)[1]
        pypdf_metadata[row_name]['File Key'] = row_name
        pypdf_metadata[row_name]['Last Modified'] = obj['LastModified']
        pypdf_metadata[row_name]['ETag'] = obj['ETag']
        pypdf_metadata[row_name]['Size'] = obj['Size']
        pypdf_metadata[row_name]['S3 Link'] = f"https://{bucket_name}.s3.amazonaws.com/{file_key}"
        pypdf_metadata[row_name]['File Type'] = file_key.split('.')[-1].lower()

# Combine PyPDF and Grobid metadata into rows
rows = []
for row_name, grobid_info in grobid_metadata.items():
    row = {}
    row.update(grobid_info)
    row.update(pypdf_metadata.get(row_name, {}))
    rows.append(row)

# Save metadata to CSV file
with open(output_csv_file, 'w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['File Key', 'Last Modified (Grobid)', 'ETag (Grobid)', 'Size (Grobid)', 'S3 Link (Grobid)', 'File Type (Grobid)',
                  'Last Modified (PyPDF)', 'ETag (PyPDF)', 'Size (PyPDF)', 'S3 Link (PyPDF)', 'File Type (PyPDF)']
    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write header row
    csv_writer.writeheader()

    # Write metadata for each row
    for row in rows:
        csv_writer.writerow({
            'File Key': row.get('File Key', ''),
            'Last Modified (Grobid)': row.get('Last Modified', ''),
            'ETag (Grobid)': row.get('ETag', ''),
            'Size (Grobid)': row.get('Size', ''),
            'S3 Link (Grobid)': row.get('S3 Link', ''),
            'File Type (Grobid)': row.get('File Type', ''),
            'Last Modified (PyPDF)': row.get('Last Modified', ''),
            'ETag (PyPDF)': row.get('ETag', ''),
            'Size (PyPDF)': row.get('Size', ''),
            'S3 Link (PyPDF)': row.get('S3 Link', ''),
            'File Type (PyPDF)': row.get('File Type', ''),
        })

print(f"Metadata saved to {output_csv_file}")


Metadata saved to ../sample_output/metadata_output.csv
